# Property Appraisal Comp Recommendation System

--- 




## Cell 1: Import Required Libraries


In [120]:
import pandas as pd
import re  # For pattern matching in text
from dateutil import parser  # For parsing various date formats
from datetime import datetime
from rapidfuzz import process, fuzz



## Cell 2: Load Cleaned Data

We'll start by loading our cleaned subjects, comps, and candidates tables that were saved during data cleaning.

In [121]:
# Load cleaned data from CSV files
subjects_df = pd.read_csv('datasets/data/subjects_raw.csv')
comps_df = pd.read_csv('datasets/data/comps_raw.csv')
candidates_df = pd.read_csv('datasets/data/candidates_raw.csv')

print("✅ Data loaded!")
print(f"Subjects:   {len(subjects_df)}")
print(f"Comps:      {len(comps_df)}")
print(f"Candidates: {len(candidates_df)}")


✅ Data loaded!
Subjects:   88
Comps:      264
Candidates: 9820


## Cell 3 - 7: Summarize Unique Values for Selected Features

This function helps you **quickly explore all unique values** for just the columns you care about, in any DataFrame (subjects, comps, candidates).

**How it works:**
- Pass your DataFrame and a list of column names you want to inspect.
- Returns a summary table showing:
  - The feature (column) name
  - All unique values found in that column
  - The total number of unique values
- If a column isn’t found, it displays "(Column missing in DataFrame)" for clarity.

---

**Example usage:**
```python
subjects_uni_df = summarize_selected_unique_features(subjects_df, subject_cols)
comps_uni_df = summarize_selected_unique_features(comps_df, comp_cols)
candidates_uni_df = summarize_selected_unique_features(candidates_df, candidate_cols)


In [122]:
def summarize_selected_unique_features(df, columns):
    """
    Shows all unique values for selected columns in a DataFrame.
    Args:
        df (pd.DataFrame): The DataFrame to summarize.
        columns (list): List of column names to check.
    Returns:
        pd.DataFrame: Summary table of unique values per selected feature.
    """
    feature_names = []
    unique_vals = []
    num_uniques = []
    
    for col in columns:
        if col in df.columns:
            uniques = df[col].unique()
            feature_names.append(col)
            unique_vals.append(list(uniques))
            num_uniques.append(len(uniques))
        else:
            feature_names.append(col)
            unique_vals.append(["(Column missing in DataFrame)"])
            num_uniques.append(0)
    
    summary_df = pd.DataFrame({
        "Feature": feature_names,
        "Unique_Values": unique_vals,
        "Num_Unique": num_uniques
    })
    
    pd.set_option('display.max_colwidth', None)
    return summary_df


In [123]:
# List of columns you want to see (from your message above)
subject_cols = [
    "effective_date", "lot_size_sf", "structure_type", "style", "room_total",
    "num_beds", "gla", "num_baths", "condition"
]

comp_cols = [
    "distance_to_subject", "prop_type", "stories", "sale_date", "lot_size",
    "condition", "gla", "room_count", "bed_count", "bath_count"
]

candidate_cols = [
    "bedrooms", "gla", "property_sub_type", "structure_type", "style",
    "levels", "room_count", "full_baths", "half_baths", "lot_size_sf",
    "close_date", "latitude", "longitude"
]

# Run the function for each DataFrame
subjects_uni_df = summarize_selected_unique_features(subjects_df, subject_cols)
comps_uni_df = summarize_selected_unique_features(comps_df, comp_cols)
candidates_uni_df = summarize_selected_unique_features(candidates_df, candidate_cols)


## Standardize Dates to YYYY-MM-DD Format

This function converts date strings from different formats (like `"Apr/11/2025"` or `"2025-01-13"`) into a standard format: **YYYY-MM-DD**.

**Key points:**
- Handles both `"Apr/11/2025"` and `"2025-01-13"` formats.
- Returns `None` for missing or invalid dates.
- Prints a warning if a date can't be parsed (helps you debug messy data).

---
**Example usage:**
```python
test_dates = ['Apr/11/2025', 'Oct/25/2024', '2025-01-13', None, 'bad_date']
for date in test_dates:
    print(f"{date} → {standardize_date(date)}")


In [124]:
def clean_date(date_string):
    """
    Cleans date strings and converts them to a standard format.
    
    Examples:
    - 'Apr/11/2025' → '2025-04-11'
    - 'Oct/25/2024' → '2024-10-25'
    - nan → None
    
    Parameters:
    - date_string: The date value to clean (could be string or NaN)
    
    Returns:
    - Standardized date string or None
    """
    # Check if the value is missing (NaN or None)
    if pd.isna(date_string) or date_string is None:
        return None
    
    try:
        # Use dateutil parser to handle various formats automatically
        parsed_date = parser.parse(str(date_string))
        # Return in standard YYYY-MM-DD format
        return parsed_date.strftime('%Y-%m-%d')
    except:
        # If parsing fails, return None
        print(f"Could not parse date: {date_string}")
        return None

# Test the function
test_dates = ['Apr/11/2025', 'Oct/25/2024', '2025-01-13', None]
for date in test_dates:
    print(f"{date} → {clean_date(date)}")

Apr/11/2025 → 2025-04-11
Oct/25/2024 → 2024-10-25
2025-01-13 → 2025-01-13
None → None


## Clean Area/Numeric Strings with Units

This function **extracts and standardizes numeric area values** from messy strings with units.  
It's designed to handle values like `"1500 SqFt"`, `"78 SqM"`, `"1.25 Acres"`, dimension strings like `"49' x 119'"`, and even hybrid formats like `"50' x 118' / 5,900 sf"`.

**How it works:**
- **If both a dimension and an explicit area are present**, it uses the area (e.g., `"50' x 118' / 5,900 sf"` returns `5900`).
- Handles common area units: **SqFt, SqM, Acres** (all returned in square feet).
- Removes commas, plus/minus signs, and ignores N/A values.
- For plain numbers (no unit), assumes square feet.
- Returns `None` if the value is missing or can't be parsed.

**Example usage:**
```python
test_values = [
    "1500 SqFt", "78 SqM", "1.25 Acres", "n/a", "3,555.5",
    "50' x 118' / 5,900 sf", "49' x 119'", "60' x 110' / 6,600 sf", "82700", "58 SqM"
]
for val in test_values:
    print(f"{val} → {clean_numeric_with_units(val)}")


In [125]:
def clean_numeric_with_units(value):
    """
    Extracts numeric values from strings with units and returns area in sqft.
    - If both a dimension and explicit area are given, prefers the area (e.g., '50\' x 118\' / 5,900 sf' returns 5900).
    - Handles 'SqFt', 'SqM', 'Acre', dimension strings, commas, +/-.
    - Returns float in sqft or None.
    """
    if pd.isna(value) or value is None:
        return None

    value_str = str(value).lower().strip()
    if value_str in ['n/a', 'na', 'nan', '', 'n/a condominium', 'n/a-condo land']:
        return None
    value_str = value_str.replace(',', '').replace('+/-', '')

    # First, check for explicit numeric area in the string (e.g., '/ 5900 sf')
    area_match = re.findall(r'(\d+\.?\d*)\s*(?:sq ?ft|sf|sq\.? ?ft|s\.?f\.?)', value_str)
    if area_match:
        # If multiple, get the biggest (more likely to be the correct one)
        return float(max([float(x) for x in area_match]))

    # If not, check for dimension: e.g., 50' x 118'
    dim_match = re.search(r'(\d+\.?\d*)\s*[\'ft]?\s*[x×]\s*(\d+\.?\d*)\s*[\'ft]?', value_str)
    if dim_match:
        width = float(dim_match.group(1))
        length = float(dim_match.group(2))
        return width * length

    # Check for acres
    if 'acre' in value_str or 'ac' in value_str:
        acre_match = re.search(r'(\d+\.?\d*)', value_str)
        if acre_match:
            return float(acre_match.group(1)) * 43560

    # Check for sqm
    if 'sqm' in value_str or 'sq m' in value_str:
        sqm_match = re.search(r'(\d+\.?\d*)', value_str)
        if sqm_match:
            return float(sqm_match.group(1)) * 10.764

    # Check for plain number (assume sqft)
    number_match = re.search(r'(\d+\.?\d*)', value_str)
    if number_match:
        return float(number_match.group(1))

    return None

# Test
test_values = [
    "1500 SqFt", "78 SqM", "1.25 Acres", "n/a", "3,555.5",
    "50' x 118' / 5,900 sf", "49' x 119'", "60' x 110' / 6,600 sf", "486 SQ M", "0.5ac"
]
for val in test_values:
    print(f"{val} → {clean_numeric_with_units(val)}")


1500 SqFt → 1500.0
78 SqM → 839.592
1.25 Acres → 54450.0
n/a → None
3,555.5 → 3555.5
50' x 118' / 5,900 sf → 5900.0
49' x 119' → 5831.0
60' x 110' / 6,600 sf → 6600.0
486 SQ M → 5231.304
0.5ac → 21780.0


## Clean and Standardize Room Count Values

This function, `clean_room_count`, cleans up room count strings and converts them to a single float value.

**How it works:**
- Handles simple numbers (`'6' → 6.0`)
- Handles composite counts (`'6+3' → 9.0`)
- Ignores missing, empty, or N/A values
- Returns `None` if the value can't be converted

**Example usage:**
```python
test_rooms = ['6', '6+3', '12+4', '15', None, 'n/a', '8+2']
for room in test_rooms:
    print(f"{room} → {clean_room_count(room)}")


In [126]:
def clean_room_count(value):
    """
    Cleans room count values.

    Examples:
    - '6' → 6.0
    - '6+3' → 9.0 (e.g., main rooms + den/bonus)
    - '8+2' → 10.0
    - nan, 'n/a', '' → None

    Parameters:
    - value: The room count value

    Returns:
    - Float total room count or None
    """
    if pd.isna(value) or value is None:
        return None

    value_str = str(value).strip()

    # Check for empty or n/a values
    if value_str.lower() in ['n/a', 'na', '', 'nan']:
        return None

    # Handle "X+Y" format (e.g., "6+3")
    if '+' in value_str:
        parts = value_str.split('+')
        try:
            total = sum(float(part.strip()) for part in parts)
            return total
        except:
            return None

    # Try to convert directly to float
    try:
        return float(value_str)
    except:
        return None

# Test the function
test_rooms = ['6', '6+3', '12+4', '15', None, 'n/a', '8+2']
for room in test_rooms:
    print(f"{room} → {clean_room_count(room)}")

6 → 6.0
6+3 → 9.0
12+4 → 16.0
15 → 15.0
None → None
n/a → None
8+2 → 10.0


## Clean and Standardize Bathroom Count Values

The `clean_bathroom_count` function standardizes various messy bathroom count formats and converts them to a single float value (where half baths count as 0.5).

**How it works:**
- Supports formats like:
  - `'2:0'` or `'2:1'` (full:half bath notation)
  - `'2F 1H'` or `'3F'` (X full, Y half notation)
  - `'2 Full/1Half'` or similar (case-insensitive)
  - Simple numbers like `'3'`
- Missing/empty/N/A values return `None`
- Returns a **float** with half baths counted as `0.5` (e.g., `2:1` → `2.5`)

**Example usage:**
```python
test_bathrooms = ['2:0', '2:1', '2F 1H', '3F', '1:2', '2 Full/1Half', '3', None]
for bath in test_bathrooms:
    print(f"{bath} → {clean_bathroom_count(bath)}")


In [127]:
def clean_bathroom_count(value):
    """
    Cleans bathroom count values with various formats.
    
    Examples:
    - '2:0' → 2.0 (2 full, 0 half)
    - '2:1' → 2.5 (2 full, 1 half)
    - '2F 1H' → 2.5 (2 full, 1 half)
    - '3F' → 3.0 (3 full)
    - '1:1' → 1.5
    
    Parameters:
    - value: The bathroom count value
    
    Returns:
    - Float number of bathrooms (half baths count as 0.5) or None
    """
    if pd.isna(value) or value is None:
        return None
    
    value_str = str(value).upper().strip()

    value_str = value_str.replace('P', 'H')
    
    # Check for empty or n/a values
    if value_str.lower() in ['n/a', 'na', '', 'nan']:
        return None
    
    # Handle "X:Y" format (full:half)
    if ':' in value_str:
        parts = value_str.split(':')
        try:
            full = float(parts[0].strip())
            half = float(parts[1].strip()) if len(parts) > 1 else 0
            return full + (half * 0.5)
        except:
            return None
    
    # Handle "XF YH" format (X full, Y half)
    if 'F' in value_str or 'H' in value_str:
        full_match = re.search(r'(\d+)\s*F', value_str)
        half_match = re.search(r'(\d+)\s*H', value_str)
        
        full = float(full_match.group(1)) if full_match else 0
        half = float(half_match.group(1)) if half_match else 0
        
        return full + (half * 0.5)
    
    # Handle "X Full/Y Half" format
    if 'FULL' in value_str or 'HALF' in value_str:
        full_match = re.search(r'(\d+)\s*FULL', value_str)
        half_match = re.search(r'(\d+)\s*HALF', value_str)
        
        full = float(full_match.group(1)) if full_match else 0
        half = float(half_match.group(1)) if half_match else 0
        
        return full + (half * 0.5)
    
    # Try direct conversion
    try:
        return float(value_str)
    except:
        return None

# Test the function
test_bathrooms = ['2:0', '2:1', '2F 1H', '3F', '1:2', '2F1P', '3', None]
for bath in test_bathrooms:
    print(f"{bath} → {clean_bathroom_count(bath)}")

2:0 → 2.0
2:1 → 2.5
2F 1H → 2.5
3F → 3.0
1:2 → 2.0
2F1P → 2.5
3 → 3.0
None → None


In [128]:
def clean_distance(value):
    """
    Cleans distance values and converts to kilometers.
    
    Examples:
    - '0.15 KM' → 0.15
    - '0.21 km' → 0.21
    - '.05 km' → 0.05
    
    Parameters:
    - value: The distance value
    
    Returns:
    - Float distance in kilometers or None
    """
    if pd.isna(value) or value is None:
        return None
    
    value_str = str(value).lower().strip()
    
    # Remove 'km' and spaces
    value_str = value_str.replace('km', '').strip()
    
    try:
        return float(value_str)
    except:
        return None

# Test the function
test_distances = ['0.15 KM', '0.21 km', '.05 km', None]
for dist in test_distances:
    print(f"{dist} → {clean_distance(dist)}")

0.15 KM → 0.15
0.21 km → 0.21
.05 km → 0.05
None → None


## Property Type Mapping

This dictionary maps messy or alternate property type names to our standard set of property types (`CANONICAL_TYPES`).  
Any value not in this list will be set to `None`.

In [159]:
manual_map = {
    # Detached houses
    "detached": "Detached",
    "detached single family": "Detached",
    "single family": "Detached",
    "single family residence": "Detached",
    "rural resid": "Detached",
    "rural residential": "Detached",
    "agriculture": "Detached",
    "farm": "Detached",
    "freehold": "Detached",
    "mobile": "Detached",
    "mobile home": "Detached",
    "mobiletrailer": "Detached",
    "mobile trailer": "Detached",

    # Semi-detached and Link homes
    "semi-detached": "Semi Detached",
    "semi detached": "Semi Detached",
    "semi detached (half duplex)": "Semi Detached",
    "link": "Semi Detached",

    # Townhouses and similar
    "townhouse": "Townhouse",
    "freehold townhouse": "Townhouse",
    "row/townhouse": "Townhouse",
    "row townhouse": "Townhouse",
    "row unit": "Townhouse",
    "row unit 2 storey": "Townhouse",
    "row unit 3 storey": "Townhouse",
    "stacked": "Townhouse",
    "stacked townhouse": "Townhouse",

    # Condos
    "condo apt": "Condominium",
    "condo townhouse": "Condominium",
    "condo apartment": "Condominium",
    "condo/apt unit": "Condominium",
    "condo unit": "Condominium",
    "common element condo": "Condominium",
    "apartment": "Condominium",

    # Duplexes, Triplexes, Fourplexes
    "duplex": "Duplex",
    "over-under": "Duplex",
    "over under": "Duplex",
    "duplex up/down": "Duplex",
    "full duplex": "Duplex",
    "triplex": "Triplex",
    "fourplex": "Fourplex",
    "4 plex": "Fourplex",
}

CANONICAL_TYPES = [
    "Townhouse", "Detached", "Condominium", "Semi Detached",
    "High Rise Apartment", "Low Rise Apartment", "Duplex", "Triplex", "Fourplex"
]

## Rapid Fuzzy Function

This function maps a property type string to a standardized value using our manual mapping:

- Checks for an **exact match** in the mapping dictionary.
- If no exact match, uses **fuzzy string matching** (`rapidfuzz`) to find the closest match above the `score_cutoff` threshold.
- Returns the standardized type if found, otherwise returns `None`.


#### **Example Usage**
Suppose you have some messy property type values in your dataset:

```python
values = ["detachd", "row unit", "condo aprtment", "semi det", "moblie home"]

detachd         -> Detached
row unit        -> Townhouse
condo aprtment  -> Condominium
semi det        -> Semi Detached
moblie home     -> Detached



In [160]:
def get_manual_type_fuzzy(value, mapping, score_cutoff=80):
    """
    Returns mapped type if a close match exists in the mapping dict, else None.
    Uses rapidfuzz for fuzzy string matching.
    """
    if not value:
        return None
    value = str(value).lower().strip()
    # Exact match
    if value in mapping:
        return mapping[value]
    # Fuzzy match
    best, score, _ = process.extractOne(
        value, list(mapping.keys()), scorer=fuzz.ratio
    )
    if score >= score_cutoff:
        return mapping[best]
    return None

## Cononical check function 


In [161]:
def canonical_check(mapped_type):
    """Return mapped_type if in canonical list, else None"""
    if mapped_type in CANONICAL_TYPES:
        return mapped_type
    return None

## Property Type Standardization

The `standardize_property_type` function combines `property_sub_type` and `structure_type` from each property record and returns a single, standardized property type.

#### How it works:
- **Prefers `property_sub_type`** if available, otherwise uses `structure_type`.
- **Manual & fuzzy matching:** First tries to map the value to a standard type using our manual dictionary (with fuzzy matching for typos and close matches).
- **Canonical check:** Only returns the type if it’s in the approved `CANONICAL_TYPES` list.
- **Substring & keyword logic:** For complex or combined values, it looks for canonical type substrings or key terms (e.g., "condo", "duplex") to determine the best match.
- **Returns `None`** if the value cannot be matched to a canonical property type.

This ensures all property types are consistently categorized for analysis and modeling.


In [162]:
def standardize_property_type(row):
    """
    Combines property_sub_type and structure_type intelligently.
    Always returns a value from CANONICAL_TYPES or None.
    """
    sub_type = row.get('property_sub_type', None)
    struct_type = row.get('structure_type', None)
    sub_type = str(sub_type).strip().lower() if pd.notna(sub_type) and sub_type not in [None, ''] else None
    struct_type = str(struct_type).strip().lower() if pd.notna(struct_type) and struct_type not in [None, ''] else None

    # Prefer sub_type if both exist
    value = sub_type if sub_type else struct_type
    if not value:
        return None

    # Try manual_type_map (exact + fuzzy)
    mapped_type = get_manual_type_fuzzy(value, manual_map, score_cutoff=80)
    result = canonical_check(mapped_type)
    if result is not None:
        return result

    # Try canonical type substrings
    for canon in CANONICAL_TYPES:
        if canon.lower() in value:
            return canon

    # Keyword-based logic (catch broad classes)
    if "condo" in value:
        return "Condominium"
    if "townhouse" in value or "row unit" in value:
        return "Townhouse"
    if "duplex" in value:
        return "Duplex"
    if "triplex" in value:
        return "Triplex"
    if "fourplex" in value or "4 plex" in value:
        return "Fourplex"
    if "semi" in value:
        return "Semi Detached"
    if "detached" in value or "single family" in value or "mobile" in value or "farm" in value:
        return "Detached"

    # Fallback: None (doesn't match canonical)
    return None

# ---- TEST CASES ----
test_rows = [
    {'property_sub_type': 'Freehold Townhouse', 'structure_type': None},
    {'property_sub_type': None, 'structure_type': 'Detached, 2-Storey'},
    {'property_sub_type': None, 'structure_type': None},
    {'property_sub_type': 'Duplex', 'structure_type': 'Detached'},
    {'property_sub_type': 'Condo Townhouse', 'structure_type': 'Condo Townhouse'},
    {'property_sub_type': 'Triplex', 'structure_type': 'Semi-Detached'},
    {'property_sub_type': 'Apartment', 'structure_type': ''},
    {'property_sub_type': '', 'structure_type': 'MobileHome'},
    {'property_sub_type': 'Semi Detached', 'structure_type': 'Duplex'},
    {'property_sub_type': 'Bungalow', 'structure_type': 'Detached, Bungalow'},
 
]

for i, row in enumerate(test_rows, 1):
    print(f"Test {i}: {row} → {standardize_property_type(row)}")


Test 1: {'property_sub_type': 'Freehold Townhouse', 'structure_type': None} → Townhouse
Test 2: {'property_sub_type': None, 'structure_type': 'Detached, 2-Storey'} → Detached
Test 3: {'property_sub_type': None, 'structure_type': None} → None
Test 4: {'property_sub_type': 'Duplex', 'structure_type': 'Detached'} → Duplex
Test 5: {'property_sub_type': 'Condo Townhouse', 'structure_type': 'Condo Townhouse'} → Condominium
Test 6: {'property_sub_type': 'Triplex', 'structure_type': 'Semi-Detached'} → Triplex
Test 7: {'property_sub_type': 'Apartment', 'structure_type': ''} → Condominium
Test 8: {'property_sub_type': '', 'structure_type': 'MobileHome'} → Detached
Test 9: {'property_sub_type': 'Semi Detached', 'structure_type': 'Duplex'} → Semi Detached
Test 10: {'property_sub_type': 'Bungalow', 'structure_type': 'Detached, Bungalow'} → None


### Story/Style Mapping

`STORY_GROUPS` lists our standard house style types.

`STOREY_MAP` converts messy or alternate style values into those standard groups.

Values not found in the map become `None`.


In [163]:
STORY_GROUPS = [
    "Bungalow",
    "Bungalow Raised",
    "1 Storey",
    "1.5 Storey",
    "2 Storey",
    "2.5 Storey",
    "3 Storey",
    "3+ Storey",
    "Split Level",
    "Bi-Level"
]

STOREY_MAP = {
    # Bungalow types
    "bungalow": "Bungalow",
    "bungalow raised": "Bungalow Raised",
    "bungalow(1 storey)": "Bungalow",
    "bungaloft": "Bungalow",   # Optionally, you could keep this separate
    "bun": "Bungalow",         # Typo, treat as bungalow

    # Single storey
    "1 storey": "1 Storey",
    "1 level": "1 Storey",
    "1": "1 Storey",
    "one": "1 Storey",
    "single level apartment": "1 Storey",
    "1 Storey/Apt": "1 Storey",
    "one level": "1 Storey",

    # 1.5 Storey
    "1.5 storey": "1.5 Storey",
    "1 1/2 storey": "1.5 Storey",
    "1 and half storey": "1.5 Storey",
    "1.5 level": "1.5 Storey",
    "1 3/4 storey": "1.5 Storey",

    # 2 Storey
    "2 storey": "2 Storey",
    "two story": "2 Storey",
    "2 level": "2 Storey",
    "attached-si": "2 Storey",    # Often attached 2 storey
    "2 storey split": "2 Storey", # Or could be Split Level
    "two": "2 Storey",
    "2": "2 Storey",

    # 2.5 Storey
    "2.5 storey": "2.5 Storey",
    "2 1/2 storey": "2.5 Storey",
    "2.5 level": "2.5 Storey",

    # 3 Storey and up
    "3 storey": "3 Storey",
    "3-storey": "3 Storey",
    "3 level": "3 Storey",
    "3+ storey": "3+ Storey",
    "3 plus stories": "3+ Storey",
    "3 (or more) storey": "3+ Storey",
    "3": "3 Storey",
    "3.0": "3 Storey",
    "three or more": "3+ Storey",
    "3 level side split": "Split Level",   # Likely split

    # 4/5+ Storey (usually not single family, but handle for completeness)
    "4 level split": "Split Level",
    "5 level split": "Split Level",
    "5 level": "Split Level",

    # Split levels
    "split level": "Split Level",
    "split entry": "Split Level",
    "sidesplit": "Split Level",
    "side split": "Split Level",
    "sidesplit 3": "Split Level",
    "sidesplit 4": "Split Level",
    "backsplit": "Split Level",
    "backsplit 3": "Split Level",
    "backsplit 4": "Split Level",
    "multi-level": "Split Level",
    "multi level unit": "Split Level",
    "multi level": "Split Level",

    # Bi-Level
    "bi-level": "Bi-Level",
    "bi level": "Bi-Level",
}




### `clean_story_group` Function

This function standardizes story/style values for properties:

- Cleans and lowercases the input.
- Maps the value to a standard group using `STOREY_MAP` (with fuzzy matching).
- If not found, checks for a standard group as a substring in the value.
- Returns the standard group name, or `None` if no match is found.


In [164]:
def clean_story_group(value):
    """
    Cleans up story/style values and groups into standard STORY_GROUPS for subject/comps.
    """
    if not value or pd.isna(value):
        return None
    value = str(value).lower().strip()
    mapped = get_manual_type_fuzzy(value, STOREY_MAP)
    if mapped in STORY_GROUPS:
        return mapped
    # Substring fallback (catch e.g. "something 2 Storey", etc.)
    for group in STORY_GROUPS:
        if group.lower() in value:
            return group
    return None

In [165]:
##-----------Test case---------##

test_story_values = [
    "2 Storey", "1.5 Storey", "Bungalow", "1 Storey", "4 Level Split",
    "2-storey", "Bungalow Raised", "2.5 Storey", "one level",
    "3 Plus Stories", "Split Level", "3+ Storey", "1 level", "Bun", '1'
]

print("Original".ljust(20), "→", "Grouped As")
for val in test_story_values:
    print(val.ljust(20), "→", clean_story_group(val))


Original             → Grouped As
2 Storey             → 2 Storey
1.5 Storey           → 1.5 Storey
Bungalow             → Bungalow
1 Storey             → 1 Storey
4 Level Split        → Split Level
2-storey             → 2 Storey
Bungalow Raised      → Bungalow Raised
2.5 Storey           → 2.5 Storey
one level            → 1 Storey
3 Plus Stories       → 3+ Storey
Split Level          → Split Level
3+ Storey            → 3+ Storey
1 level              → 1 Storey
Bun                  → Bungalow
1                    → 1 Storey


## Standardize Storey/Style for Candidate Properties

This function cleans and standardizes the story/style type for candidate properties by:
- Preferring the 'style' field, then checking 'levels' if needed.
- Using our mapping and fallback logic to ensure values are grouped into standard STORY_GROUPS.
- Returns None if no valid mapping is found.

This helps make all storey/style data consistent for modeling and analysis.


In [166]:
def standardize_candidate_storey(row):
    """
    Cleans and standardizes story/style info for candidate properties.
    Prefers 'style', then 'levels', else returns None.
    Uses clean_story_group for the mapping logic.
    """
    style = row.get('style', None)
    levels = row.get('levels', None)

    # Try style first
    result = clean_story_group(style)
    if result:
        return result

    # If style fails, try levels
    result = clean_story_group(levels)
    if result:
        return result

    # If both fail, return None
    return None


In [167]:
# --- Example Test Cases for standardize_candidate_storey ---

test_rows = [
    {'style': '2 Storey, Attached-Si', 'levels': '2 Storey, Attached-Si'},       # Should match '2 Storey'
    {'style': 'Bungalow-Raised', 'levels': None},                                # Should match 'Bungalow Raised'
    {'style': None, 'levels': '1 1/2 Storey'},                                   # Should match '1.5 Storey'
    {'style': '3-Storey', 'levels': '3 Level'},                                  # Should match '3 Storey'
    {'style': 'Split Entry', 'levels': ''},                                      # Should match 'Split Level'
    {'style': None, 'levels': None},                                             # Should return None
    {'style': 'Backsplit 4', 'levels': 'Split Level'},                           # Should match 'Split Level'
    {'style': 'Apartment-High-Rise', 'levels': None},                            # Should return None (not in story group)
    {'style': '', 'levels': 'One Level'},                                        # Should match '1 Storey'
    {'style': 'Something Unusual', 'levels': 'Something Else'},                  # Should return None
    {'style': 'Bungaloft', 'levels': ''},                                        # Should match 'Bungalow'
    {'style': '2', 'levels': ''},                                                # Should match '2 Storey' (if mapped in STOREY_MAP)
    {'style': 'Bi-Level', 'levels': None},                                       # Should match 'Bi-Level'
]

print("---- STANDARDIZATION TEST CASES ----")
for i, row in enumerate(test_rows, 1):
    result = standardize_candidate_storey(row)
    print(f"Test {i:2}: {row} --> {result}")


---- STANDARDIZATION TEST CASES ----
Test  1: {'style': '2 Storey, Attached-Si', 'levels': '2 Storey, Attached-Si'} --> 2 Storey
Test  2: {'style': 'Bungalow-Raised', 'levels': None} --> Bungalow Raised
Test  3: {'style': None, 'levels': '1 1/2 Storey'} --> 1.5 Storey
Test  4: {'style': '3-Storey', 'levels': '3 Level'} --> 3 Storey
Test  5: {'style': 'Split Entry', 'levels': ''} --> Split Level
Test  6: {'style': None, 'levels': None} --> None
Test  7: {'style': 'Backsplit 4', 'levels': 'Split Level'} --> Split Level
Test  8: {'style': 'Apartment-High-Rise', 'levels': None} --> None
Test  9: {'style': '', 'levels': 'One Level'} --> 1 Storey
Test 10: {'style': 'Something Unusual', 'levels': 'Something Else'} --> None
Test 11: {'style': 'Bungaloft', 'levels': ''} --> Bungalow
Test 12: {'style': '2', 'levels': ''} --> 2 Storey
Test 13: {'style': 'Bi-Level', 'levels': None} --> Bi-Level


In [168]:
def clean_all_property_data(subjects_df, comps_df, candidates_df):
    """
    Master function to clean all property datasets with all the cleaning functions.
    
    Parameters:
    - subjects_df: DataFrame with subject properties
    - comps_df: DataFrame with comp properties  
    - candidates_df: DataFrame with candidate properties
    
    Returns:
    - Tuple of (cleaned_subjects_df, cleaned_comps_df, cleaned_candidates_df)
    """
    
    # Make copies to avoid modifying originals
    subjects_clean = subjects_df.copy()
    comps_clean = comps_df.copy()
    candidates_clean = candidates_df.copy()
    
    print("Starting comprehensive data cleaning...")
    print("-" * 50)
    
    # ========== CLEAN SUBJECTS ==========
    print("\n📋 Cleaning SUBJECTS data...")
    
    # Clean dates
    if 'effective_date' in subjects_clean.columns:
        subjects_clean['effective_date_clean'] = subjects_clean['effective_date'].apply(clean_date)
        print(f"  ✓ Cleaned effective_date")
    
    # Clean lot size
    if 'lot_size_sf' in subjects_clean.columns:
        subjects_clean['lot_size_sf_clean'] = subjects_clean['lot_size_sf'].apply(clean_numeric_with_units)
        print(f"  ✓ Cleaned lot_size_sf")
    
    # Clean GLA (Gross Living Area)
    if 'gla' in subjects_clean.columns:
        subjects_clean['gla_clean'] = subjects_clean['gla'].apply(clean_numeric_with_units)
        print(f"  ✓ Cleaned gla")
    
    # Clean room counts
    if 'room_total' in subjects_clean.columns:
        subjects_clean['room_total_clean'] = subjects_clean['room_total'].apply(clean_room_count)
        print(f"  ✓ Cleaned room_total")
    
    # Clean bedrooms
    if 'num_beds' in subjects_clean.columns:
        subjects_clean['num_beds_clean'] = subjects_clean['num_beds'].apply(clean_room_count)
        print(f"  ✓ Cleaned bed_count")
    
    # Clean bathrooms
    if 'num_baths' in subjects_clean.columns:
        subjects_clean['num_baths_clean'] = subjects_clean['num_baths'].apply(clean_bathroom_count)
        print(f"  ✓ Cleaned num_baths")
    
    # Standardize property type (using structure_type for subjects)
    if 'structure_type' in subjects_clean.columns:
        subjects_clean['property_type_clean'] = subjects_clean.apply(
            lambda row: standardize_property_type({'property_sub_type': None, 'structure_type': row['structure_type']}), 
            axis=1
        )
        print(f"  ✓ Standardized property type")
    
    # Clean story/style
    if 'style' in subjects_clean.columns:
        subjects_clean['style_clean'] = subjects_clean['style'].apply(clean_story_group)
        print(f"  ✓ Cleaned style")
    
    # ========== CLEAN COMPS ==========
    print("\n📋 Cleaning COMPS data...")
    
    # Clean distance
    if 'distance_to_subject' in comps_clean.columns:
        comps_clean['distance_to_subject_clean'] = comps_clean['distance_to_subject'].apply(clean_distance)
        print(f"  ✓ Cleaned distance_to_subject")
    
    # Clean dates
    if 'sale_date' in comps_clean.columns:
        comps_clean['sale_date_clean'] = comps_clean['sale_date'].apply(clean_date)
        print(f"  ✓ Cleaned sale_date")
    
    # Clean lot size
    if 'lot_size' in comps_clean.columns:
        comps_clean['lot_size_clean'] = comps_clean['lot_size'].apply(clean_numeric_with_units)
        print(f"  ✓ Cleaned lot_size")
    
    # Clean GLA
    if 'gla' in comps_clean.columns:
        comps_clean['gla_clean'] = comps_clean['gla'].apply(clean_numeric_with_units)
        print(f"  ✓ Cleaned gla")
    
    # Clean room count
    if 'room_count' in comps_clean.columns:
        comps_clean['room_count_clean'] = comps_clean['room_count'].apply(clean_room_count)
        print(f"  ✓ Cleaned room_count")
    
    # Clean bedrooms
    if 'bed_count' in comps_clean.columns:
        comps_clean['bed_count_clean'] = comps_clean['bed_count'].apply(clean_room_count)
        print(f"  ✓ Cleaned bed_count")
        
    # Clean bathrooms
    if 'bath_count' in comps_clean.columns:
        comps_clean['bath_count_clean'] = comps_clean['bath_count'].apply(clean_bathroom_count)
        print(f"  ✓ Cleaned bath_count")
    
    # Standardize property type
    if 'prop_type' in comps_clean.columns:
        comps_clean['property_type_clean'] = comps_clean.apply(
            lambda row: standardize_property_type({'property_sub_type': row.get('prop_type'), 'structure_type': None}), 
            axis=1
        )
        print(f"  ✓ Standardized property type")
    
    # Clean stories
    if 'stories' in comps_clean.columns:
        comps_clean['stories_clean'] = comps_clean['stories'].apply(clean_story_group)
        print(f"  ✓ Cleaned stories")
    
    # ========== CLEAN CANDIDATES ==========
    print("\n📋 Cleaning CANDIDATES data...")
    
    # Clean dates
    if 'close_date' in candidates_clean.columns:
        candidates_clean['close_date_clean'] = candidates_clean['close_date'].apply(clean_date)
        print(f"  ✓ Cleaned close_date")
    
    # Clean lot size
    if 'lot_size_sf' in candidates_clean.columns:
        candidates_clean['lot_size_sf_clean'] = candidates_clean['lot_size_sf'].apply(clean_numeric_with_units)
        print(f"  ✓ Cleaned lot_size_sf")
    
    # Clean GLA
    if 'gla' in candidates_clean.columns:
        candidates_clean['gla_clean'] = candidates_clean['gla'].apply(clean_numeric_with_units)
        print(f"  ✓ Cleaned gla")
    
    # Clean room count
    if 'room_count' in candidates_clean.columns:
        candidates_clean['room_count_clean'] = candidates_clean['room_count'].apply(clean_room_count)
        print(f"  ✓ Cleaned room_count")
    

    # Clean bedroom 
    if 'bedrooms' in candidates_clean.columns:
        candidates_clean['bedrooms_clean'] = candidates_clean['bedrooms'].apply(clean_room_count)
        print(f"  ✓ Cleaned bedrooms")

    # Clean bathrooms (handle full and half separately, then combine)
    if 'full_baths' in candidates_clean.columns and 'half_baths' in candidates_clean.columns:
        candidates_clean['full_baths_clean'] = pd.to_numeric(candidates_clean['full_baths'], errors='coerce')
        candidates_clean['half_baths_clean'] = pd.to_numeric(candidates_clean['half_baths'], errors='coerce')
        candidates_clean['total_baths_clean'] = (
            candidates_clean['full_baths_clean'].fillna(0) + 
            (candidates_clean['half_baths_clean'].fillna(0) * 0.5)
        )
        print(f"  ✓ Cleaned and combined full_baths and half_baths")
    
    # Standardize property type
    if 'property_sub_type' in candidates_clean.columns or 'structure_type' in candidates_clean.columns:
        candidates_clean['property_type_clean'] = candidates_clean.apply(standardize_property_type, axis=1)
        print(f"  ✓ Standardized property type")
    
    # Standardize story/style
    if 'style' in candidates_clean.columns or 'levels' in candidates_clean.columns:
        candidates_clean['style_clean'] = candidates_clean.apply(standardize_candidate_storey, axis=1)
        print(f"  ✓ Standardized style/levels")
    
    # ========== SUMMARY STATISTICS ==========
    print("\n📊 Cleaning Summary:")
    print("-" * 50)
    
    # Subjects summary
    print(f"\nSUBJECTS ({len(subjects_clean)} records):")
    for col in ['effective_date_clean', 'lot_size_sf_clean', 'gla_clean', 'room_total_clean', 
                'num_baths_clean', 'property_type_clean', 'style_clean']:
        if col in subjects_clean.columns:
            non_null = subjects_clean[col].notna().sum()
            pct = (non_null / len(subjects_clean)) * 100
            print(f"  - {col}: {non_null}/{len(subjects_clean)} ({pct:.1f}% populated)")
    
    # Comps summary
    print(f"\nCOMPS ({len(comps_clean)} records):")
    for col in ['distance_to_subject_clean', 'sale_date_clean', 'lot_size_clean', 'gla_clean',
                'room_count_clean', 'bath_count_clean', 'property_type_clean', 'stories_clean']:
        if col in comps_clean.columns:
            non_null = comps_clean[col].notna().sum()
            pct = (non_null / len(comps_clean)) * 100
            print(f"  - {col}: {non_null}/{len(comps_clean)} ({pct:.1f}% populated)")
    
    # Candidates summary
    print(f"\nCANDIDATES ({len(candidates_clean)} records):")
    for col in ['close_date_clean', 'lot_size_sf_clean', 'gla_clean', 'room_count_clean',
                'total_baths_clean', 'property_type_clean', 'style_clean']:
        if col in candidates_clean.columns:
            non_null = candidates_clean[col].notna().sum()
            pct = (non_null / len(candidates_clean)) * 100
            print(f"  - {col}: {non_null}/{len(candidates_clean)} ({pct:.1f}% populated)")
    
    print("\n✅ Data cleaning complete!")
    
    return subjects_clean, comps_clean, candidates_clean


# Example usage:
cleaned_subjects, cleaned_comps, cleaned_candidates = clean_all_property_data(    
    subjects_df, comps_df, candidates_df
)

# Optional: Save cleaned data
cleaned_subjects.to_csv('datasets/data/subjects_cleaned.csv', index=False)
cleaned_comps.to_csv('datasets/data/comps_cleaned.csv', index=False)  
cleaned_candidates.to_csv('datasets/data/candidates_cleaned.csv', index=False)

Starting comprehensive data cleaning...
--------------------------------------------------

📋 Cleaning SUBJECTS data...
  ✓ Cleaned effective_date
  ✓ Cleaned lot_size_sf
  ✓ Cleaned gla
  ✓ Cleaned room_total
  ✓ Cleaned bed_count
  ✓ Cleaned num_baths
  ✓ Standardized property type
  ✓ Cleaned style

📋 Cleaning COMPS data...
  ✓ Cleaned distance_to_subject
  ✓ Cleaned sale_date
  ✓ Cleaned lot_size
  ✓ Cleaned gla
  ✓ Cleaned room_count
  ✓ Cleaned bed_count
  ✓ Cleaned bath_count
  ✓ Standardized property type
  ✓ Cleaned stories

📋 Cleaning CANDIDATES data...
  ✓ Cleaned close_date
  ✓ Cleaned lot_size_sf
  ✓ Cleaned gla
  ✓ Cleaned room_count
  ✓ Cleaned bedrooms
  ✓ Cleaned and combined full_baths and half_baths
  ✓ Standardized property type
  ✓ Standardized style/levels

📊 Cleaning Summary:
--------------------------------------------------

SUBJECTS (88 records):
  - effective_date_clean: 88/88 (100.0% populated)
  - lot_size_sf_clean: 72/88 (81.8% populated)
  - gla_clean: 88

In [ ]:
def clean_all_property_data(subjects_df, comps_df, candidates_df):
    """
    Master function to clean all property datasets with consistent clean column names.
    """
    # Make copies
    subjects_clean = subjects_df.copy()
    comps_clean = comps_df.copy()
    candidates_clean = candidates_df.copy()
    
    print("Starting comprehensive data cleaning...")
    print("-" * 50)
    
    # ========== CLEAN SUBJECTS ==========
    print("\n📋 Cleaning SUBJECTS data...")
    subjects_clean['effective_date_clean'] = subjects_clean['effective_date'].apply(clean_date)
    subjects_clean['lot_size_clean'] = subjects_clean['lot_size_sf'].apply(clean_numeric_with_units)
    subjects_clean['gla_clean'] = subjects_clean['gla'].apply(clean_numeric_with_units)
    subjects_clean['bedrooms_clean'] = subjects_clean['num_beds'].apply(clean_room_count)
    subjects_clean['bathrooms_clean'] = subjects_clean['num_baths'].apply(clean_bathroom_count)
    subjects_clean['property_type_clean'] = subjects_clean.apply(
        lambda row: standardize_property_type({'property_sub_type': None, 'structure_type': row['structure_type']}), axis=1)
    subjects_clean['stories_clean'] = subjects_clean['style'].apply(clean_story_group)
    subjects_clean['style_clean'] = subjects_clean['style'].apply(clean_story_group)  # Optionally keep for style

    # ========== CLEAN COMPS ==========
    print("\n📋 Cleaning COMPS data...")
    comps_clean['distance_to_subject_clean'] = comps_clean['distance_to_subject'].apply(clean_distance)
    comps_clean['sale_date_clean'] = comps_clean['sale_date'].apply(clean_date)
    comps_clean['lot_size_clean'] = comps_clean['lot_size'].apply(clean_numeric_with_units)
    comps_clean['gla_clean'] = comps_clean['gla'].apply(clean_numeric_with_units)
    comps_clean['bedrooms_clean'] = comps_clean['bed_count'].apply(clean_room_count)
    comps_clean['bathrooms_clean'] = comps_clean['bath_count'].apply(clean_bathroom_count)
    comps_clean['property_type_clean'] = comps_clean.apply(
        lambda row: standardize_property_type({'property_sub_type': row.get('prop_type'), 'structure_type': None}), axis=1)
    comps_clean['stories_clean'] = comps_clean['stories'].apply(clean_story_group)
    comps_clean['style_clean'] = comps_clean['stories'].apply(clean_story_group)  # Optionally keep for style

    # ========== CLEAN CANDIDATES ==========
    print("\n📋 Cleaning CANDIDATES data...")
    candidates_clean['close_date_clean'] = candidates_clean['close_date'].apply(clean_date)
    candidates_clean['lot_size_clean'] = candidates_clean['lot_size_sf'].apply(clean_numeric_with_units)
    candidates_clean['gla_clean'] = candidates_clean['gla'].apply(clean_numeric_with_units)
    candidates_clean['bedrooms_clean'] = candidates_clean['bedrooms'].apply(clean_room_count)
    # Bathrooms: combine full & half into a single clean value
    candidates_clean['full_baths_clean'] = pd.to_numeric(candidates_clean['full_baths'], errors='coerce')
    candidates_clean['half_baths_clean'] = pd.to_numeric(candidates_clean['half_baths'], errors='coerce')
    candidates_clean['bathrooms_clean'] = (
        candidates_clean['full_baths_clean'].fillna(0) +
        (candidates_clean['half_baths_clean'].fillna(0) * 0.5)
    )
    candidates_clean['property_type_clean'] = candidates_clean.apply(standardize_property_type, axis=1)
    candidates_clean['stories_clean'] = candidates_clean.apply(standardize_candidate_storey, axis=1)
    candidates_clean['style_clean'] = candidates_clean['style'].apply(standardize_candidate_storey)  # Optionally keep for style

    # ========== SUMMARY STATISTICS ==========
    print("\n📊 Cleaning Summary:")
    print("-" * 50)
    print(f"\nSUBJECTS ({len(subjects_clean)} records):")
    for col in ['effective_date_clean', 'lot_size_clean', 'gla_clean', 'bedrooms_clean', 'bathrooms_clean', 'property_type_clean', 'stories_clean']:
        non_null = subjects_clean[col].notna().sum()
        pct = (non_null / len(subjects_clean)) * 100
        print(f"  - {col}: {non_null}/{len(subjects_clean)} ({pct:.1f}% populated)")
    
    print(f"\nCOMPS ({len(comps_clean)} records):")
    for col in ['distance_to_subject_clean', 'sale_date_clean', 'lot_size_clean', 'gla_clean', 'bedrooms_clean', 'bathrooms_clean', 'property_type_clean', 'stories_clean']:
        non_null = comps_clean[col].notna().sum()
        pct = (non_null / len(comps_clean)) * 100
        print(f"  - {col}: {non_null}/{len(comps_clean)} ({pct:.1f}% populated)")
    
    print(f"\nCANDIDATES ({len(candidates_clean)} records):")
    for col in ['close_date_clean', 'lot_size_clean', 'gla_clean', 'bedrooms_clean', 'bathrooms_clean', 'property_type_clean', 'stories_clean']:
        non_null = candidates_clean[col].notna().sum()
        pct = (non_null / len(candidates_clean)) * 100
        print(f"  - {col}: {non_null}/{len(candidates_clean)} ({pct:.1f}% populated)")
    
    print("\n✅ Data cleaning complete!")
    return subjects_clean, comps_clean, candidates_clean


In [ ]:
cleaned_subjects, cleaned_comps, cleaned_candidates = clean_all_property_data(
    subjects_df, comps_df, candidates_df
)
# Save as needed:
cleaned_subjects.to_csv('datasets/data/subjects_cleaned.csv', index=False)
cleaned_comps.to_csv('datasets/data/comps_cleaned.csv', index=False)  
cleaned_candidates.to_csv('datasets/data/candidates_cleaned.csv', index=False)
